[Tensor Switching Networks](https://arxiv.org/pdf/1610.10087v1.pdf) (sort of) generalise relus. The intuition derives from writing a relu layer's output as:
$$
    X_l = \left(H(W_{l}X_{l-1}) \otimes X_{l-1} \odot W_{l}\right) \times \mathbf{1}^{n_0}
$$

Where $X_i$ is the activation for layer $i$, $W_i$ the weights, $H(\cdot)$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function) applied elementwise,
$\odot$ is a kind of broadcasted tensor-matrix Hadamard where we do it for each slice of the tensor. The vector of ones at the end squishes it back down the right shape.

Tensor switching networks simplify slightly by skipping the projection at the end:
$$
    Z_l = H(W_lX_{l-1}) \otimes Z_{l-1}
$$
ensuring $Z_0 = X_0$ is the input to the network. Essentially this just gets gated in complex ways as the tensor grows (with a new dimension for each layer). Finally, just flatten the thing and send it into a standard layer.

There are two issues with this:
1. the tensor gets very large
    - because it adds a whole dimension at each layer (it is likely to be sparse and what is there is just copies).
1. it is impossible to train the weights with normal backprop
    - because $H$ has zero gradients (where it is defined).

The authors offer the following solutions:
1. nothing
2. some crazy schemes including:
    - just use learn only the last layer
    - use nearly the gradients you would get if it were a normal relu net
    - pretty much changing everything into a different kind of network (which is interesting in its own right)
    
    
Here we propose to investigate 2 ideas to solve these issues:
1. use the tensor train format
2. use [this](https://arxiv.org/pdf/1602.02830v3.pdf) method of pretending to back-propagate
   through binarisation.

#### Tensor train tensor products.

What does it mean to do the tensor product of a vector with a tensor in the tt-format.
Let $A$ be a tensor in the tensor-train with elements
$$
    a(i_1, i_2, \ldots, i_d) = G_1[i_1]G_2[i_2]\cdots G_d[i_d]
$$
Consider $C = b \otimes A$ for some $b \in \mathbb{R}^{m}$. This should be the same as making $m$ copies of $A$ and multiplying each copy by different elements of $b$. Then
$$
    c(i_m, i_1, i_2, \ldots, i_d) = b_{i_m} G_1[i_1]G_2[i_2]\cdots G_d[i_d]
$$

So this is equivalent to adding dummy dimensions to make 
$b \in \mathbb{R}^{1\times m\times 1}$ and sticking it on the end of the tensor train.
This seems intuitive on the surface -- we increase the dimensions by adding a new core.
It also starts to make it feel quite suspiciously like we're going to end up with a
pretty standard looking deep net if we apply this to the above tensor switching net.

#### _Tensor train tensor switching network_

We start with an input vector $Z_0 = X_0$, and for each layer $l$ compute:
$$
X_l = H(W_lX_{l-1}) \\
Z_l = X_l \otimes Z_{l-1}
$$
Treating $Z_0$ as the first core of a decomposition,
this gives the following for an element of the penultimate layer:
$$
Z_L(i_L, i_{L-1}, \ldots i_1) = X_L(i_L)X_{L-1}(i_{L-1}) \cdots X_1(i_1).
$$
This is what we would expect. As the X are 0,1 for a value to propagate its way to the output,
every layer has to let it through (but there are exponentially many chances for
this to happen). Curiously, we haven't actually changed anything here -- the decomposition
at this stage buys us nothing as the cores being added have to have tt-rank 1.

The output layer is linear and needs to see all elements of $Z_L$. As we've gone to all the
trouble of keeping $Z_L$ in a tensor train, it makes sense to try and avoid blowing it all
back up right at the end, especially as it turns out to be a very low-rank tensor. The answer
here is to realise what we're actually doing -- we want a linear operation to reduce a tensor
a vector. We want to therefore contract away all the dimensions except one and then project
that to the right size. This should be able to be realised in a single tensor-tensor product
(which should turn out to be equivalent to flattening and doing a matrix product). If we 
recognise the tensor-tensor nature of this final layer, we can use a low-rank tensor train
for the output weights and avoid horrible exponential sizes.

Introduce a final weight tensor $U \in \mathbb{R}^{t, n_L \times \cdots \times n_1}$ with a
dimension per layer and an extra output dimension of size $t$. 
Denoting the output of the network $\hat{Y}$, a single element should be
$$
\hat{Y}_k = \sum_{i_L, \ldots i_1} U(k, i_L, \ldots, i_1)Z_L(i_L, \ldots, i_1)
$$ (which affirms the intuition that it is the same as flattening an using matrices).

Recalling the definition of $Z_l$ and letting $U$ be in the tt-format with $L+1$ cores $G_i$,
$$
\hat{Y}_k = \sum_{i_L, \ldots i_1} G_{L+1}(k)G_{L}(i_L)\cdots G_{1}(i_1) X_L(i_L)\cdots X_1(i_1).
$$ Seeing as the $X_j(i_j)$ are all in fact scalars, we can rearrange:
$$
\hat{Y}_k = G_{L+1}(k) \left(\sum_{i_L}G_{L}(i_L) X_L(i_L)\right) \cdots \left(\sum_{i_1}G_1(i_1)\right).
$$ As $G_{L+1}(k)$ is a row vector, $G_{1}(i_1)$ is a column vector and the rest of the $G$
slices are matrices the shapes work out nicely.

We should be able now to define the whole network in one go -- the _analysis_ weights $U$ get
folded in to each layer. This means each layer $l$ essentially has two outputs: the running
product of the reduced $G_l$s (which we term $Y_l$ 
and the most recent activations $X_l$. The activations are used
to compute the next set of activations, which are used to reduce the current $G_l$ which is in
turn used to produce the output $Y_l$. Formally:
$$
X_l = H(W_lX_{l-1}) \\
Y_l = \left(\sum_{i_l}G_l(i_l)X_l(i_l)\right)Y_{l-1}
$$
and finally
$$
\hat{Y} = G_{L+1} Y_L
$$ (technically $G_{L+1}$ is a 3-tensor, but one dimension is 1 so here we treat it as a matrix).

Probably it will turn out to be a good plan to add biases to the linear transformations.

##### training

We have a network, it's kind of nice. At each stage, we make some binary masks and use them to
select slices of the $G$ to combine. The $G$ have a nice linear progression all the way to the
input. The second key issue is still unresolved: how to train the $W_l$. Clearly the $G_l$
should be fine with normal backprop, but the Heaviside step function has zero gradients.
Therefore $W_l$ will never learn and we are doomed.

Tsai, Saxe and Cox propose few methods, the only one which appears to remain relevant involves
computing the pseudoinverse of the gradients of auxiliary variables to derive a
pseudo-gradient for the problematic variables. This seems somewhat unpleasant, so it might
be preferable to avoid it if possible.

An alternative approach for backpropagating through binarising input functions is presented
by [Courbariaux et al.](https://arxiv.org/pdf/1602.02830v3.pdf) (although not for the first
time). They are using a sign function
$$
    q = S(r) = \begin{cases} 1 & r > 0 \\
    -1 & \text{otherwise}
    \end{cases}
$$ to binarize
and use a pseudo-gradient referred to as the _straight through_ estimator (note that it
is not an estimator -- the gradients are 0 or undefined, this does not require estimation).
Using $S^*$ in the place of $S'$ to denote this false gradient, this approach defines:
$$
    S^*(r) = 1_{|r| \leq 1}
$$
That is to say: a gradient of 1 unless the input is outside a certain range. This is precisely
the gradient of the hard tanh function $h(r) = max(-1, min(1, r))$.

The step function we are intending to use is very similar to the sign function above -- the
only difference is that the lowest value it outputs is zero. This suggests we may succeed with
a similar pseudo-gradient, so it is worth a shot. As the threshold is in the same place, there
is probably no need to adjust the above.

An alternative is to relax the decision function into something that does have a 
(sub-)gradient. One answer might be a sigmoid, although these are notoriously poor in deep
feed-forward nets. An answer which retains a lot of desirable properties might be a variation
on the hard tanh above: $s(r) = max(0, min(1, r))$. This is a kind of hard sigmoid, without
the scaling usually applied. The scaling and shifting of the input is omitted for two reasons:
firstly, if it is helpful the network should be able to learn it and secondly it maintains the
same boundary as the heaviside step function -- any input less than zero will output zero.

This hard sigmoid has a non-zero gradient when the input is between 0 and 1, so we may be able
to learn something. Let's go ahead and implement both of these and compare.

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# had to implement the fake gradient activation as its own op
from ops import binary_connect
from functools import partial

bc_01 = partial(binary_connect, min_=0.0)

from rnndatasets import sequentialmnist as data

In [33]:
def hard_sigmoid(activations):
    return tf.maximum(0.0, tf.minimum(1.0, activations))

def ttts_layer(previous_activations, previous_output, hidden_units, tensor_rank,
               nonlin=hard_sigmoid, scope=None):
    """Makes a layer as described above. Tensor rank gets to only be a single integer
    because previous_output should be [batch_size, previous_rank]"""
    with tf.variable_scope(scope or 'ttts_layer'):
        # first compute activations
        input_shape = previous_activations.get_shape()[1].value
        weights = tf.get_variable('weights', shape=[input_shape, hidden_units])
        # give the biases a high-ish init and it will start off basically linear
        biases = tf.get_variable('biases', shape=[hidden_units],
                                 initializer=tf.constant_initializer(1.0))
        acts = tf.nn.bias_add(tf.matmul(previous_activations, weights), biases)
        acts = nonlin(acts)
        
        # now get the tt-core of the output tensor and do a bilinear product with
        # previous_output and acts
        prev_rank = previous_output.get_shape()[1].value
        # we're going to do this all in terms of flattenings, so that we can use the
        # nice big matmuls. The downside is there's a bit of exra storage
        core = tf.get_variable('core', shape=[prev_rank*hidden_units, tensor_rank])
        # if we take the outer product of previous_output and hidden_units then
        # we can just multiply by the above. Regrettably this can be quite large.
        # There's possibly a better way, but that can wait
        outer = tf.batch_matmul(tf.expand_dims(previous_output, 2),
                                tf.expand_dims(acts, 1))
        outer = tf.reshape(outer, [-1, prev_rank*hidden_units])
        output = tf.matmul(outer, core)
        
    return acts, output

def get_full_net(input_var, num_outputs, depth, width, rank, nonlin=hard_sigmoid,
                 scope=None):
    """Gets the full network. Not terribly flexible, all layers are the same
    width and all of the TT cores are the same rank (except where they have
    to be 1)"""
    with tf.variable_scope(scope or 'ttttttttsnettt'):
        activations, outputs = input_var, input_var
        for layer in range(depth):
            activations, outputs = ttts_layer(activations, outputs,
                                              width, rank, nonlin,
                                              'layer_{}'.format(layer))
        # now the last layer is just a linear transform
        # should be rank -> num_outputs
        with tf.variable_scope('output_layer'):
            weights = tf.get_variable('weights', [rank, num_outputs])
            biases = tf.get_variable('biases', [num_outputs])
            result = tf.nn.bias_add(tf.matmul(outputs, weights), biases)
        return result
    
def count_params(scope):
    return np.sum(
        [np.prod([dim for dim in var.get_shape().as_list()]) 
         for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)])

In [35]:
BATCH_SIZE = 50

tf.reset_default_graph()
input_pl = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 728], name='inputs')
target_pl = tf.placeholder(tf.int32, shape=[BATCH_SIZE], name='targets')

with tf.variable_scope('hs_net') as scope:
    hs_net_out = get_full_net(input_pl, 10, 5, 10, 10, scope=scope)
    print('hard sigmoid: {} parameters'.format(count_params(scope.name)))

hard sigmoid: 84640 parameters
